<a href="https://colab.research.google.com/github/SanghunOh/study_machinglearning/blob/main/codes/sanghunoh/reports/Term_2_MLP_MPIE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Connect Drive

In [1]:
path_root = '/content/drive/MyDrive/datas/'

# 작업 경로 설정
import os
os.chdir(path_root)
!pwd

/content/drive/MyDrive/Colab Notebooks/00.study/10.knou/datas


In [2]:
# !unzip -o ./mpie_30_shuffle.zip -d ./mpie_30_shuffle

In [3]:
!ls

dataCh4_7.mat		matlab_iris_shuffle.mat
dataCh4_7.zip		mpie_30_shuffle
digitimages_binary.zip	mpie_30_shuffle.zip
iris_shuffle.mat	number_images
iris.zip		results_MLP_20222805080015.pickle


###Load Dataset

In [4]:
path = path_root + 'mpie_30_shuffle/'

import pandas as pd

def loadDatasetFromCSV(_datafilename, _labelfilename, label_column):
  _feature_csv = pd.read_csv(path + _datafilename, dtype=np.float32, header=None) # image features of train data

  _label_csv = pd.read_csv(path + _labelfilename, dtype=np.float32, header=None) # labels of train data
  _label = _label_csv[label_column] # get label you want

  return _feature_csv, _label

In [5]:
import numpy as np

In [6]:
def preprocessDataset(label_column):
  # train
  train_feature_reshape, train_label = loadDatasetFromCSV('Traindata.csv', 'Trainlabel.csv', label_column)
  # test
  validation_feature_reshape, validation_label = loadDatasetFromCSV('Testdata.csv', 'Testlabel.csv', label_column)

  return train_feature_reshape, train_label, validation_feature_reshape, validation_label

In [7]:
train_feature, train_label, validation_feature, validation_label = preprocessDataset(1)
type(train_feature), train_feature.shape, type(train_label), train_label.shape, validation_feature.shape, validation_label.shape

(pandas.core.frame.DataFrame,
 (18777, 1024),
 pandas.core.series.Series,
 (18777,),
 (5086, 1024),
 (5086,))

####check model

In [8]:
import tensorflow as tf
from tensorflow.keras import layers

class_cnt = len(np.unique(train_label))
# class_cnt

model = tf.keras.Sequential()
model.add(layers.Dense(256, input_dim=1024, kernel_initializer='glorot_uniform', activation='relu')) 
model.add(layers.BatchNormalization())
model.add(layers.Dense(256, input_dim=1024, kernel_initializer='glorot_uniform', activation='relu')) 
model.add(layers.BatchNormalization())
model.add(layers.Dense(class_cnt, activation='softmax')) 

model.compile(loss='sparse_categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

_epochs = 10
model.fit(train_feature, train_label)

587/587 [==============================] - 7s 10ms/step - loss: 1.1351 - accuracy: 0.5933


In [9]:
# input_layer = layers.Input(shape=(1024,))  # input: 1024 nodes
# x = tf.keras.layers.Flatten(name='Flatten')(input_layer)
# x = layers.Activation('relu')(layers.Dense(256, input_dim=1024)(x))  # hidden: 256 nodes
# x = layers.BatchNormalization()(x)
# y = layers.Activation('softmax')(layers.Dense(class_cnt)(x))    # number of output nodes : class_no (it changes depend on label you selected)
# model = tf.keras.Model(x, y)
# model.compile(loss='sparse_categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

# model.fit(train_feature, train_label)

###MPL CNN with BatchNormalization

In [10]:
import numpy as np

class_cnt = len(np.unique(train_label))
class_cnt

6

In [11]:
import tensorflow as tf
from tensorflow.keras import layers

def make_model(_class_cnt, _hidden_layers=1, dense_count=256, _kernel_initializer='glorot_uniform', _activation='relu', _optimizer='SGD', batch_normal=False, epochs=2, label_class=0):
  _model = tf.keras.Sequential()
  _model.add(layers.Dense(dense_count, input_dim=1024, kernel_initializer=_kernel_initializer, activation=_activation)) 
  if batch_normal:
    _model.add(layers.BatchNormalization())
  for idx in range(_hidden_layers):
    _model.add(layers.Dense(dense_count, kernel_initializer=_kernel_initializer, activation=_activation)) 
    if batch_normal:
      _model.add(layers.BatchNormalization())
  _model.add(layers.Dense(_class_cnt, activation='softmax')) 

  _model.compile(loss='sparse_categorical_crossentropy', optimizer=_optimizer, metrics=['accuracy'])

  return _model
model = make_model(_class_cnt=6)    
# tf.keras.utils.plot_model(model, show_shapes=True, show_dtype=True, expand_nested=True, show_layer_activations=True) #layer_range=?, 

###callback function for fit time

In [12]:
import time

class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)
TimeHistory()

### fit model

In [13]:
rows = train_feature
# _batch_size = int(len(list(rows)) / (len(list(rows))/2))   # Just Check Model params quickly
_batch_size = len(list(rows))

def model_fit(_param, _dataset):
  tf.keras.backend.clear_session()

  (_train_feature, _train_label, _validation_feature, _validation_label) = _dataset
  _param['_class_cnt'] = len(np.unique(_train_label))
  print(_param['epochs'], _param['_class_cnt'])

  model = make_model(**_param)
  time_callback = TimeHistory()
  history = model.fit(_train_feature, _train_label, epochs=_param['epochs'], validation_data=(_validation_feature, _validation_label), callbacks=[time_callback], steps_per_epoch=_batch_size)
  # history = model.fit(_train_feature, _train_label, epochs=_epochs, callbacks=[time_callback])
  execution_time = sum(time_callback.times)
  return model, history, execution_time, _param
dataset = preprocessDataset(1)  
param = {'_hidden_layers':1, 'dense_count':256, '_kernel_initializer':'glorot_uniform', '_activation':'relu', 'batch_normal':False, 'epochs':2}
model_fit(param, dataset)  

2 6
Epoch 1/2
2/2 [==============================] - 1s 744ms/step - loss: 1.7879 - accuracy: 0.1820 - val_loss: 1.5974 - val_accuracy: 0.5319
Epoch 2/2
2/2 [==============================] - 1s 591ms/step - loss: 1.5460 - accuracy: 0.5646 - val_loss: 1.4977 - val_accuracy: 0.5387


(<keras.engine.sequential.Sequential at 0x7f76fabcc890>,
 2.2066006660461426,
 {'_activation': 'relu',
  '_class_cnt': 6,
  '_hidden_layers': 1,
  '_kernel_initializer': 'glorot_uniform',
  'batch_normal': False,
  'dense_count': 256,
  'epochs': 2})

### fit with multi params

In [14]:
# params = [
#           {'_hidden_layers':0, 'dense_count':64, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':5},
#           {'_hidden_layers':0, 'dense_count':64, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':50},
#           {'_hidden_layers':0, 'dense_count':64, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':100},
#           {'_hidden_layers':0, 'dense_count':256, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':5},
#           {'_hidden_layers':0, 'dense_count':256, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':50},
#           {'_hidden_layers':0, 'dense_count':256, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':100},
#           {'_hidden_layers':0, 'dense_count':64, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':5},
#           {'_hidden_layers':0, 'dense_count':64, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':50},
#           {'_hidden_layers':0, 'dense_count':64, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':100},
#           ]

In [15]:
# param 6
params = list()
param_key_list = list(param.keys())
_hidden_layers_list = [0, 5, 10]
dense_count_list = [64, 256]
_kernel_initializer_list = [None, 'glorot_uniform']
_activation_list = ['sigmoid', 'relu']
_optimizer_list= ['SGD', 'Adam']
batch_normal_list = [False, True]
epochs_list = [10, 50, 100, 500]

for _hidden_layers in _hidden_layers_list:
  for dense_count in dense_count_list:
    for _kernel_initializer in _kernel_initializer_list:
      for _activation in _activation_list:
        for _optimizer in _optimizer_list:
          for batch_normal in batch_normal_list:
            for epochs in epochs_list:
              param_dict = dict()
              param_dict['_hidden_layers'] = _hidden_layers
              param_dict['dense_count'] = dense_count
              param_dict['_kernel_initializer'] = _kernel_initializer
              param_dict['_activation'] = _activation
              param_dict['_optimizer'] = _optimizer
              param_dict['batch_normal'] = batch_normal
              param_dict['epochs'] = epochs
              # print(param_dict)
              params.append(param_dict)
param_dict              

In [ ]:
label_classes = [0, 3, 5]
for label in label_classes:
  dataset = preprocessDataset(label)
  results = list()          
  for idx, param in enumerate(params):
    param['label_class'] = label
    print('-'*5 +'[ '+ str(idx) + ' ]'+ '-'*5 + str(param))
    results.append(model_fit(param, dataset))


-----[ 0 ]-----{'_hidden_layers': 0, 'dense_count': 64, '_kernel_initializer': None, '_activation': 'sigmoid', '_optimizer': 'SGD', 'batch_normal': False, 'epochs': 10, 'label_class': 0}
10 30
Epoch 1/10
2/2 [==============================] - 2s 956ms/step - loss: 3.5724 - accuracy: 0.0564 - val_loss: 3.5672 - val_accuracy: 0.0588
Epoch 2/10
2/2 [==============================] - 1s 812ms/step - loss: 3.5659 - accuracy: 0.0570 - val_loss: 3.5610 - val_accuracy: 0.0584
Epoch 3/10
2/2 [==============================] - 1s 722ms/step - loss: 3.5597 - accuracy: 0.0573 - val_loss: 3.5549 - val_accuracy: 0.0566
Epoch 4/10
2/2 [==============================] - 1s 665ms/step - loss: 3.5537 - accuracy: 0.0574 - val_loss: 3.5491 - val_accuracy: 0.0568
Epoch 5/10
2/2 [==============================] - 1s 537ms/step - loss: 3.5479 - accuracy: 0.0577 - val_loss: 3.5436 - val_accuracy: 0.0568
Epoch 6/10
2/2 [==============================] - 1s 676ms/step - loss: 3.5423 - accuracy: 0.0576 - val_los

In [ ]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%Y%d%m%H%M%S")
print("date and time =", dt_string)

import pickle
# save
with open(f'results_MLP_{dt_string}.pickle', 'wb') as f:
    pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)